## TD METHODS

In DP methods, we found optimal policies. But, we used ```transition_map``` to do that. This is quite impractical. There are many reasons for that:

- We don't know all the states we can visit
- Environment dynamics can be unknown
- Iterating over all states can be computationally infeasible

In this notebook, we will be practicing over TD methods. Instead of calculating the exact value, we will get samples and estimate the value over samples. One approach that comes into our minds, when we talk about sampling, is **Monte Carlo** method. MC method is the first algorithm we will be implementing.

**MC** estimation and control is pretty simple. We sample trajectories from the environment by following a policy $\pi$ and calculate the returns of each state we visit.

**Question 1)** How can you ensure that we visit (sample) enough states to find an optimal policy?

#### Environment

We will be using a new environment called **Warehouse** where an agent tries to match each item with the corresponding box. Building a transition map for this environment would be painful. Luckily, we don't need to build that in TD methods. The ```worldmap``` is given below (you can modify this for fun but, we will be using the one below to evaluate).

You can run the cell below to visualize the environment.
- <span style="color:#989898">Dark gray cells</span> are impassable cells.
- <span style="color:#DADADA">light gray cells</span> are passable empty cells.
- <span style="color:#00B8FA">Blue cell</span> is the agent.
- <span style="color:#A33675"> Darker magenta cell </span> and <span style="color:#48C69F"> darker green cell </span> are items to collect. 
- <span style="color:#B34685"> Lighter magenta cell </span> and <span style="color:#58D6AF"> lighter green cell </span> are the boxes.

Pairing is also given below. A key in the pairing dictionary is a box(uppercase) and its corresponding value is a list of items(lowercase) that can be delivered to that box.

State representation is different here. Instead of giving just the position of the agent, we have 4 more additional features. These are 4 boolean values for two items and two boxes.  The boolean feature is true if the item or the box exists on the map and false otherwise. When the bucket receives the item it disappears.

In [ ]:
%load_ext autoreload
%autoreload 2

from rl_hw1.env import Warehouse

worldmap = ["#########",
            "#   #   #",
            "# c # C #",
            "#   #   #",
            "#   P   #",
            "#   #   #",
            "# b # B #",
            "#   #   #",
            "#########"]

# Buckets are Uppercase letters while balls are lowercase
# Matching is done so that the ball "b" must be carried to the bucket "B"
pairing = {
    "B": ["b"],
    "C": ["c"]
}

env = Warehouse(balls="cb", buckets="BC",
                pairing=pairing, worldmap=worldmap)
env.init_render()

**Question 2)** In the ```rl_hw1/learning/monte_carlo.py``` module, implement ```MonteCarloAgent```



Let's train the ```MonteCarloAgent``` using the training loop given below. You can tune hyperparameters(```args```) given below. ```args``` is a namedtuple built from the dictionary initialized between lines 10 and 18. 

**Question 3)** Train the ```MonteCarloAgent``` by tuning the hyperparameters. For all of your implementations (MC, Q learning, Sarsa) your agent should get at least **1.5** episodic reward(sum of rewards throughout an episode).

In [ ]:
from rl_hw1.learning import MonteCarloAgent
from collections import namedtuple
import numpy as np
import random

# Initialize agent
agent = MonteCarloAgent(nact=4)

# Hyperparameters
args = dict(
    iteration = 3000,
    gamma = 0.98,
    alpha = 0.03,
    init_eps = 0.5,
    final_eps = 0.1,
    eps_decay_rate = 0.999,
    seed = 12021,            # Current year: (10000 + 2021 = 12021) Holocene calendar
)
args = namedtuple('args', args.keys())(*args.values()) 

# Seed
np.random.seed(args.seed)
random.seed(args.seed)

# Traning loop
reward_list = []
epsilon = args.init_eps
for ix in range(args.iteration):
    epsilon = max(args.final_eps, epsilon*args.eps_decay_rate)
    reward = agent.one_epsiode_train(
        env, lambda x: agent.e_greedy_policy(x, epsilon), args.gamma, args.alpha)
    reward_list.append(reward)
    if ((ix + 1) % 50) == 0:
        print("Episode: {}, reward: {}".format(ix + 1, np.mean(reward_list[-100:])))

Now, let's try to render the trained agent. Run the followwing cells below.

In [ ]:
env = Warehouse(balls="cb", buckets="BC",
                pairing=pairing, worldmap=worldmap)
env.init_render()

In [ ]:
# You can run this cell to visualize the agent as many times as you wish.
agent.evaluate(env, render=True)

MC method waits until the episode is terminated to begin the update. But, is it possible to update the policy within every transition? Temporal Difference(TD) methods exactly aim for this. There are two popular TD methods that you will be implementing: Q Learning and SARSA.

**Question 3)** What is the difference between off-policy and on-policy algorithms?


### Q - Learning

Q learning is an off-policy algorithm that employs temporal difference value estimation.

**Question 4)** What makes Q learning an off-policy algorithm? Why don't we use importance sampling in Q learning?

**Question 5)**  In the ```rl_hw1/learning/td.py``` module, implement ```QAgent```

In [ ]:
from collections import namedtuple
import numpy as np
import random
import matplotlib.pyplot as plt

from rl_hw1.learning import QAgent, SarsaAgent


# Initiate environment
env = Warehouse(balls="cb", buckets="BC",
                pairing=pairing, worldmap=worldmap)

# Initialize agent
q_agent = QAgent(nact=4)
# Hyperparameters
args = dict(
    episodes = 1500,
    evaluate_period = 50,
    gamma = 0.98,
    alpha = 0.03,
    init_eps = 0.9,
    final_eps = 0.05,
    eps_decay_rate = 0.9995,
    seed = 12021,            # Current year: (10000 + 2021 = 12021) Holocene calendar
)
args = namedtuple('args', args.keys())(*args.values()) 

# Seed
np.random.seed(args.seed)
random.seed(args.seed)


q_rewards = q_agent.train(env, q_agent.e_greedy_policy, args)

In [ ]:
%matplotlib inline
plt.plot(np.convolve(q_rewards, np.ones(10)/10, "valid"))
plt.show()

In [ ]:
env.init_render()

In [ ]:
# You can run this cell to visualize the agent as many times as you wish.
q_agent.evaluate(env, render=True)

You implemented an off-policy TD(Temporal Difference) Learning algorithm. Now, try to implement SARSA. You can find the pseudocode in the 6th chapter of Reinforcement Learning Book (Sutton & Barta).

**Question 6)** What makes SARSA an on-policy algorithm?

**Question 7)** In the ```rl_hw1/learning/td.py``` module, implement ```SarsaAgent```

**Question 8)** Write down a training loop similar to what we did for Q learning to test your ```SarsaAgent```

In [ ]:
# Sarsa training loop
#  ______   _____   _        _
# |  ____| |_   _| | |      | |
# | |__      | |   | |      | |
# |  __|     | |   | |      | |
# | |       _| |_  | |____  | |____
# |_|      |_____| |______| |______|

In [ ]:
# Sarsa reward visaulization
#  ______   _____   _        _
# |  ____| |_   _| | |      | |
# | |__      | |   | |      | |
# |  __|     | |   | |      | |
# | |       _| |_  | |____  | |____
# |_|      |_____| |______| |______|

In [ ]:
# Sarsa visaulization
#  ______   _____   _        _
# |  ____| |_   _| | |      | |
# | |__      | |   | |      | |
# |  __|     | |   | |      | |
# | |       _| |_  | |____  | |____
# |_|      |_____| |______| |______|

**Question 9)** Compare MC and TD based methods. When would you prefer one over the other?

**Question 10)** Can you think of an algorithm that combines the advanteges of MC and TD methods. Write down a **simple**  pseudocode.

#### Approximate Methods

We have seen different ways to estimate value, however, until now we only worked on tabular representations. But almost always states are represented by vectors. In this part of the homework, you will be working on linear representations that can work with continuous state spaces.

We can use any function to represent values as long as it returns a single scalar value for a state-action pair. For simplicity, we will be using a linear function. Function approximation methods can be used in both Sarsa and Q learning. In this homework, we will be implementing function approximation with Q learning.

You can find a similar algorithm within the pseudocode at **chapter 9.3 in Reinforcement Learning Book (Sutton & Barto)**. These methods are called **semi-gradient** methods. The reason is that we do not take the gradient of the loss term(mean squared TD) with respect to both functions $Q(s,a)$ and $\max_i Q(s', a_i)$.

Please read the relevant section

**Question 11)** In tabular settings, whenever we update the value of a particular state action pair $(s, a)$, only a single value changes. What are your thoughts about the value changes in approximate settings after an update? How is it different than the tabular case? 

In this part of the homework, we will be using the **CartPole** environment. It is a continuous state, discrete action environment. You can check it out from [Open AI gym](https://gym.openai.com/envs/CartPole-v0/) for more details.

**Question 12)** We will be using a linear function. However, we still need to have some features over raw observation. Complete the implementation of the wrapper below.

In [ ]:
from gym import ObservationWrapper
from gym.spaces import Box


class FeaturedObservation(ObservationWrapper):
    """ Create feature vectors that are randomly scattered in the observation
    space. Instead of returning the raw state(observation) return features.
    The feature matrix is a KxN matrix where K denotes the number of features while N
    denotes raw state space size. Each row in the Feature matrix represents a feature
    in the state space.
    
    F(S): R^{N} -> R^{K}  
    
    Returned vector is simply a dot product between each feature vector and the
    observed state(observation).
    
    Arguments:
        - env: gym environment
        - n_features: Number of features
    """
    def __init__(self, env, n_features):
        super().__init__(env)
        self.features = np.random.uniform(
            -0.4, 0.4,
            size=(n_features, env.observation_space.shape[0]))
        _low = env.observation_space.low.min()
        _high = env.observation_space.high.max()
        _dtype = env.observation_space.dtype
        self.observation_space  = Box(low=_low,
                                      high=_high,
                                      shape=(n_features,),
                                      dtype=_dtype)

    def observation(self, observation):
        """ Return features of the obervation using matrix multiplication.
        """
        #  ______   _____   _        _
        # |  ____| |_   _| | |      | |
        # | |__      | |   | |      | |
        # |  __|     | |   | |      | |
        # | |       _| |_  | |____  | |____
        # |_|      |_____| |______| |______|

**Question 13)** In the ```rl_hw1/learning/approximate.py``` module, implement ```ApproximateAgent``` & ```ApproximateQAgent```

**Question 14)** Run the approximate q learning agent by tuning the hyperparameters (```args```) given below.

In [ ]:
from collections import namedtuple
import numpy as np
import random
import gym
import matplotlib.pyplot as plt

from rl_hw1.learning import ApproximateQAgent

# Hyperparameters
args = dict(
    episodes = 800,
    evaluate_period = 200,
    n_features = 128,
    gamma = 0.98,
    alpha = 0.001,
    init_eps = 0.9,
    final_eps = 0.05,
    eps_decay_rate = 0.99999,
    seed = 12021,            # Current year: (10000 + 2021 = 12021) Holocene calendar
)
args = namedtuple('args', args.keys())(*args.values())

# Initiate environment
env = gym.make("CartPole-v1")

# Seed
env.seed(args.seed)
np.random.seed(args.seed)
random.seed(args.seed)

# Wrap the environment
env = FeaturedObservation(env, args.n_features)

# Initialize agent
appx_q_agent = ApproximateQAgent(
    nobs = env.observation_space.shape[0],
    nact = env.action_space.n
)
    
appx_q_rewards, appx_q_losses = appx_q_agent.train(env, appx_q_agent.e_greedy_policy, args)

In [ ]:
plt.title("Approximate q learning training")
plt.xlabel("episodic reward")
plt.ylabel("reward")
plt.plot(appx_q_rewards)

Now evaluate your agent. We expect to see at least **100** episodic reward in average.

In [ ]:
# You can run this cell to visualize the agent
# If you run evaluation with render from docker it may not work!!
appx_q_agent.evaluate(env, render=True)

In [ ]:
appx_q_agent_rewards = [appx_q_agent.evaluate(env, render=False) for i in range(100)]
print("Average episodic reward: {:.4}, Deviation: {:.4}".
      format(np.mean(appx_q_agent_rewards), np.std(appx_q_agent_rewards)))